# Trigger files
This notebook is intended to create the trigger files for the eeg data analysis.

In [3]:
import copy
import datetime
import itertools
import os

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from matplotlib.ticker import FormatStrFormatter

import pandas as pd
import pyxdf
import seaborn as sns
from scipy.signal import find_peaks
from tqdm.notebook import tqdm

In [152]:
# path to data stored
r_path = "data"
# path to store trigger data
t_path = r_path + '/triggers'
# path to store eye-tracking data
e_path = r_path + '/eye_tracking'

files = os.listdir(r_path)  # get all files from the folder "data"
files.sort()  # sort them alphabetically
recordings = {}
for i, file in enumerate(files):  # store and display all files
    if file.endswith('.xdf'):
        created = os.path.getmtime(f"data/{file}")  # creation timestamp
        created = datetime.datetime.fromtimestamp(created)  # translate as datetime
        created = created.strftime("%d.%m.%Y %H:%M")  # arrange it
        recordings[i] = {"file": file, "created": created}

files = [f.split(".")[0] for f in files]
print("Included:")
display(recordings)

Included:


{1: {'file': '39_room1_251022.xdf', 'created': '25.10.2022 16:40'},
 2: {'file': '40_room1_261022.xdf', 'created': '26.10.2022 16:54'}}

## 1. Load data

In [114]:
# check streams for recording 0
streams, _ = pyxdf.load_xdf(f"data/{recordings[2]['file']}")

In [115]:
# stream channel names in recording 0
s_channels = {streams[i]["info"]["name"][0]: i for i in range(len(streams))}
s_channels

{'HitPositionOnObjects': 0,
 'ImagesOrder': 1,
 'HitObjectPositions': 2,
 'Visual': 3,
 'ImageInfo': 4,
 'ValidationError': 5,
 'EyeTrackingLocal': 6,
 'HeadTracking': 7,
 'openvibeSignal': 8,
 'EyeTrackingWorld': 9,
 'HitObjectNames': 10}

In [108]:
for key, value in enumerate(s_channels):
    print(str(value))

ImagesOrder
ValidationError
HeadTracking
openvibeSignal
EyeTrackingLocal
EyeTrackingWorld
HitObjectPositions
ImageInfo
Visual
HitObjectNames
HitPositionOnObjects


In [148]:
# streams[2]['info']['desc']
streams[7]['time_series']

array([[ 0.05188722,  2.6171317 ,  0.26521954, -0.00564698, -0.0143108 ,
         0.9998816 ],
       [ 0.05189477,  2.6171064 ,  0.26518586, -0.00556182, -0.01410132,
         0.9998851 ],
       [ 0.05188295,  2.617096  ,  0.26518765, -0.00543394, -0.01407026,
         0.9998862 ],
       ...,
       [ 0.06265525,  2.620007  ,  0.3050403 , -0.08418018,  0.07387327,
         0.99370843],
       [ 0.06265999,  2.6200013 ,  0.30497307, -0.08413529,  0.07380783,
         0.9937171 ],
       [ 0.0626609 ,  2.6199858 ,  0.30492318, -0.0842477 ,  0.0739575 ,
         0.99369645]], dtype=float32)

## 2. Create dataframe from streams

In [316]:
def get_streams_data(streams, streams_keep=['ImageInfo','Visual']):
    """
    :param streams: streams after loading from .xdf file
    :param streams_keep: str. of the stream names to keep
    :return: df containing the time_stamps and stream_data as columns for each stream to keep
    """
    data = pd.DataFrame()
    for i, ch_name in enumerate(streams_keep):
        # get all current streams with their positions on the recording
        # example: {'ImagesOrder': 0, 'ValidationError': 1, 'HeadTracking': 2}
        s_channels = {streams[i]["info"]["name"][0]: i for i in range(len(streams))}
        u = s_channels[ch_name]
        # save the subject UID and append to df
        uid = streams[u]['info']['uid']
        data['uid'] = np.resize(uid,len(data))

        # check the type and length of data arrays and get only 1 value of the array
        stream_data = streams[u]['time_series']
        # double check keys on each stream to make sure they are all appended to df
        # print(f"Stream {ch_name} keys: {streams[u]['info']['desc'][0].keys()}")
        # check stram_data is of kind np.array()
        if isinstance(stream_data, (list,pd.core.series.Series,np.ndarray)):
            # access all stream names in dictionary's 'info' description
            for i, key in enumerate(streams[u]['info']['desc'][0].keys()):
                # save each dict key as column to df
                stream_data = pd.DataFrame(streams[u]['time_series'])[i]
                data[f"{key}_{ch_name}"] = stream_data

        # get timestamps and attach them as column to df
        time_stamps =  streams[u]['time_stamps']
        data = pd.concat([data, pd.DataFrame(time_stamps, columns=[f"time_stamps_{ch_name}"])], axis=1)

    return data

In [317]:
df = get_streams_data(streams, streams_keep=['ImageInfo','Visual'])
df

,uid,blockNumber_ImageInfo,imageName_ImageInfo,time_stamps_ImageInfo,cFrame_Visual,displayStatus_Visual,worldTime_Visual,time_stamps_Visual
0,dfb99d79-4595-4a0d-b346-23282e000f10,0,startMessage,1.021258e+06,1831.0,-1.0,57.157524,1.021258e+06
1,dfb99d79-4595-4a0d-b346-23282e000f10,0,startMessage,1.021258e+06,1832.0,-1.0,57.162384,1.021258e+06
2,dfb99d79-4595-4a0d-b346-23282e000f10,0,startMessage,1.021258e+06,1832.0,-1.0,57.163330,1.021258e+06
3,dfb99d79-4595-4a0d-b346-23282e000f10,0,startMessage,1.021258e+06,1832.0,-1.0,57.163502,1.021258e+06
4,dfb99d79-4595-4a0d-b346-23282e000f10,0,startMessage,1.021258e+06,1832.0,-1.0,57.163681,1.021258e+06
...,...,...,...,...,...,...,...,...
345912,dfb99d79-4595-4a0d-b346-23282e000f10,3,endMessage,1.025199e+06,347754.0,99.0,3998.594971,1.025199e+06
345913,dfb99d79-4595-4a0d-b346-23282e000f10,3,endMessage,1.025199e+06,347755.0,99.0,3998.606201,1.025199e+06
345914,dfb99d79-4595-4a0d-b346-23282e000f10,3,endMessage,1.025199e+06,347756.0,99.0,3998.617188,1.025199e+06
345915,dfb99d79-4595-4a0d-b346-23282e000f10,3,endMessage,1.025199e+06,347757.0,99.0,3998.628174,1.025199e+06


In [125]:
# quick statistics to know if all rows have same length
df.describe()
# df['HON_HitObjectNames'].isnull().sum()
# df['HON_HitObjectNames'].unique()

,time_stamps_ImageInfo,cFrame_Visual,displayStatus_Visual,worldTime_Visual,time_stamps_Visual
count,3.459170e+05,345917.000000,345917.000000,345917.000000,3.459170e+05
mean,1.023226e+06,174779.500000,2.006155,2025.562988,1.023226e+06
std,1.142137e+03,99866.382812,3.835076,1142.134766,1.142137e+03
min,1.021258e+06,1831.000000,-1.000000,57.157524,1.021258e+06
25%,1.022251e+06,88278.000000,1.000000,1050.553223,1.022251e+06
50%,1.023212e+06,174796.000000,2.000000,2011.432495,1.023212e+06
75%,1.024205e+06,261276.000000,2.000000,3004.952637,1.024205e+06
max,1.025199e+06,347758.000000,99.000000,3998.639404,1.025199e+06


In [149]:
# inspect the eye_tracking data
df_eye = get_streams_data(streams, streams_keep=['HitObjectNames', 'HitPositionOnObjects','HeadTracking','EyeTrackingWorld','EyeTrackingLocal'])
df_eye

Stream HitObjectNames keys: dict_keys(['HON'])
Stream HitPositionOnObjects keys: dict_keys(['HPOOX', 'HPOOY', 'HPOOZ'])
Stream HeadTracking keys: dict_keys(['HToriginX', 'HToriginY', 'HToriginZ', 'HTdirectionX', 'HTdirectionY', 'HTdirectionZ'])
Stream EyeTrackingWorld keys: dict_keys(['ETWTime', 'ETWoriginX', 'ETWoriginY', 'ETWoriginZ', 'ETWdirectionX', 'ETWdirectionY', 'ETWdirectionZ', 'leftBlink', 'rightBlink', 'valid'])


,UID,HON_HitObjectNames,time_stamps_HitObjectNames,HPOOX_HitPositionOnObjects,HPOOY_HitPositionOnObjects,HPOOZ_HitPositionOnObjects,time_stamps_HitPositionOnObjects,HToriginX_HeadTracking,HToriginY_HeadTracking,HToriginZ_HeadTracking,...,ETWoriginX_EyeTrackingWorld,ETWoriginY_EyeTrackingWorld,ETWoriginZ_EyeTrackingWorld,ETWdirectionX_EyeTrackingWorld,ETWdirectionY_EyeTrackingWorld,ETWdirectionZ_EyeTrackingWorld,leftBlink_EyeTrackingWorld,rightBlink_EyeTrackingWorld,valid_EyeTrackingWorld,time_stamps_EyeTrackingWorld
0,6041b0a9-1fb2-4c9b-8d36-29b6c1ee3421,DisplayCanvas,1.021275e+06,0.688816,2.983938,9.900000,1.021275e+06,0.051887,2.617132,0.265220,...,0.082040,2.615271,0.267029,0.062838,0.038180,0.997602,0.0,0.0,1.0,1.021275e+06
1,6041b0a9-1fb2-4c9b-8d36-29b6c1ee3421,DisplayCanvas,1.021275e+06,0.690930,2.991683,9.900000,1.021275e+06,0.051895,2.617106,0.265186,...,0.082048,2.615263,0.267020,0.063055,0.038982,0.997584,0.0,0.0,1.0,1.021275e+06
2,6041b0a9-1fb2-4c9b-8d36-29b6c1ee3421,DisplayCanvas,1.021275e+06,0.686294,2.992306,9.900000,1.021275e+06,0.051883,2.617096,0.265188,...,0.082061,2.615264,0.267023,0.062576,0.039047,0.997610,0.0,0.0,1.0,1.021275e+06
3,6041b0a9-1fb2-4c9b-8d36-29b6c1ee3421,DisplayCanvas,1.021275e+06,0.623313,2.981323,9.900000,1.021275e+06,0.051898,2.617072,0.265240,...,0.060661,2.614614,0.266999,0.058286,0.037988,0.997889,0.0,0.0,1.0,1.021275e+06
4,6041b0a9-1fb2-4c9b-8d36-29b6c1ee3421,DisplayCanvas,1.021275e+06,0.623996,2.980973,9.900000,1.021275e+06,0.051895,2.617085,0.265254,...,0.061041,2.614604,0.266990,0.058317,0.037952,0.997884,0.0,0.0,1.0,1.021275e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268047,6041b0a9-1fb2-4c9b-8d36-29b6c1ee3421,FixationCollider,1.025194e+06,-0.093281,1.308327,9.285572,1.025194e+06,0.062706,2.620039,0.305145,...,0.061185,2.622173,0.305193,-0.017096,-0.145415,0.993936,0.0,0.0,1.0,1.025194e+06
268048,6041b0a9-1fb2-4c9b-8d36-29b6c1ee3421,FixationCollider,1.025194e+06,-0.090902,1.307544,9.285041,1.025194e+06,0.062677,2.620066,0.305095,...,0.061382,2.622180,0.305206,-0.016855,-0.145510,0.993928,0.0,0.0,1.0,1.025194e+06
268049,6041b0a9-1fb2-4c9b-8d36-29b6c1ee3421,FixationCollider,1.025194e+06,-0.088866,1.283516,9.278251,1.025194e+06,0.062655,2.620007,0.305040,...,0.062218,2.622194,0.305289,-0.016732,-0.148252,0.993710,0.0,0.0,1.0,1.025194e+06
268050,6041b0a9-1fb2-4c9b-8d36-29b6c1ee3421,FixationCollider,1.025194e+06,-0.088808,1.282828,9.278063,1.025194e+06,0.062660,2.620001,0.304973,...,0.062758,2.622205,0.305336,-0.016785,-0.148332,0.993702,0.0,0.0,1.0,1.025194e+06


In [151]:
# df_eye.describe()
df_eye.isnull().sum()

UID                                 0
HON_HitObjectNames                  0
time_stamps_HitObjectNames          0
HPOOX_HitPositionOnObjects          0
HPOOY_HitPositionOnObjects          0
HPOOZ_HitPositionOnObjects          0
time_stamps_HitPositionOnObjects    0
HToriginX_HeadTracking              0
HToriginY_HeadTracking              0
HToriginZ_HeadTracking              0
HTdirectionX_HeadTracking           0
HTdirectionY_HeadTracking           0
HTdirectionZ_HeadTracking           0
time_stamps_HeadTracking            0
ETWTime_EyeTrackingWorld            0
ETWoriginX_EyeTrackingWorld         0
ETWoriginY_EyeTrackingWorld         0
ETWoriginZ_EyeTrackingWorld         0
ETWdirectionX_EyeTrackingWorld      0
ETWdirectionY_EyeTrackingWorld      0
ETWdirectionZ_EyeTrackingWorld      0
leftBlink_EyeTrackingWorld          0
rightBlink_EyeTrackingWorld         0
valid_EyeTrackingWorld              0
time_stamps_EyeTrackingWorld        0
dtype: int64

## 3. Create triggers
- For each initial time an image was shown, we want to keep the type of object it was (i.e., face, object, body) as a separate column.
- Additional triggers contain the rotation and distance the specific object was with respect to the player at the time the free-viewing walk took place.
- __Note:__ We want the triggers only once to denote the initial time the image was shown.

In [318]:
def create_triggers(df):
    # save the names of the image category
    df['ob_names'] =  df.apply(lambda x: x["imageName_ImageInfo"].split(".")[5] if len(x["imageName_ImageInfo"].split(".")) > 7 else '', axis=1)
    # creating the triggers for the first time an image is shown
    # check when there is a change from image, canvas, fixationCross
    df['shift'] = df['imageName_ImageInfo'].shift(1) != df['imageName_ImageInfo']
    df['time_onset'] = df.apply(lambda x: x['time_stamps_Visual'] if len(x["imageName_ImageInfo"].split(".")) > 7 and x['shift'] else '', axis=1)
    df['triggers'] = df.apply(lambda x: 'face' if x['shift'] and 'face' in x['imageName_ImageInfo'].lower()
                                               else ('body' if x['shift'] and 'npc' in x['imageName_ImageInfo'].lower()
                                               else ('object' if x['shift'] and 'rotation' in x['imageName_ImageInfo'].lower() and 'face|npc' not in x['imageName_ImageInfo'].lower()
                                               else '')), axis=1)
    # define the triggers for rotation and distance
    df['rotation'] = df.apply(lambda x: 'r_' + x["imageName_ImageInfo"].split(".")[7] if len(x["imageName_ImageInfo"].split(".")) > 7 and x['shift'] else '', axis=1)
    df['distance'] = df.apply(lambda x: 'd_' + x["imageName_ImageInfo"].split(".")[9] if len(x["imageName_ImageInfo"].split(".")) > 7 and x['shift'] else '', axis=1)
    df['block'] = df.apply(lambda x: 'b_' + str(x["blockNumber_ImageInfo"]) if len(x["imageName_ImageInfo"].split(".")) > 7 and x['shift'] else '', axis=1)
    df_sel = df[['time_onset','triggers','rotation','distance','block']]
    df_triggers = df_sel[df_sel['time_onset'] != '']
    uid = df['uid'][0]
    return df, df_triggers, uid #, total_images

In [319]:
df, df_triggers, uid = create_triggers(df)
df_triggers

,time_onset,triggers,rotation,distance,block
1726,1021276.707802,body,r_117,d_5,b_0
1873,1021278.34043,object,r_49,d_7,b_0
2019,1021279.961899,face,r_29,d_7,b_0
2189,1021281.861035,face,r_72,d_3,b_0
2345,1021283.593613,face,r_14,d_3,b_0
...,...,...,...,...,...
344807,1025186.804562,body,r_109,d_9,b_3
344967,1025188.581551,body,r_54,d_5,b_3
345148,1025190.591745,body,r_5,d_1,b_3
345294,1025192.213329,body,r_54,d_5,b_3


In [330]:
# path to save .csv with total images per block
total_img_file = os.path.join(t_path, 'total_unique_images_per_user.csv')

# progress bar format definitons
m_format = "{desc}:{bar}{percentage:3.0f}% {n_fmt}/{total_fmt} in {elapsed_s:.2f}s"
s_format = ("{desc}:{bar}{percentage:3.0f}% {n_fmt}/{total_fmt}{postfix} in {elapsed_s:.2f}s")
# main progress bar
main_bar = tqdm(
    os.listdir(r_path),
    desc="Processed",
    dynamic_ncols=True,
    mininterval=0.001,
    bar_format=m_format,
)
# for k in main_bar:
for file in main_bar:

    if file.lower().endswith('.xdf'):
        pbar = tqdm(
        range(5),
        mininterval=0.001,
        maxinterval=1,
        bar_format=s_format,)

        pbar.set_postfix(file=file)
        #### 1. Load the XDF file ####
        postfix = {"step": "1. Load the XDF file", "file": file}
        pbar.set_postfix(postfix)

        streams, _ = pyxdf.load_xdf(os.path.join(r_path, file))
        pbar.update(1)
        #### 2. Store selected stream info (only useful info)
        postfix = {"step": "2. Store selected streams info", "file": file}
        pbar.set_postfix(postfix)
        # store trigger-related stream data into df
        df = get_streams_data(streams, streams_keep=['ImageInfo','Visual'])
        # save eye_tracking related data
        df_eye = get_streams_data(streams, streams_keep=['HitObjectNames', 'HitPositionOnObjects','HeadTracking','EyeTrackingWorld','EyeTrackingLocal'])
        pbar.update(1)

        #### 3. Create triggers
        postfix = {"step": "3. Creating triggers from ImageInfo", "file": file}
        pbar.set_postfix(postfix)
        df_img, df_triggers, uid = create_triggers(df)
        pbar.update(1)

        #### 4. Save total number of images per block, per uid
        postfix = {"step": "4 Saving total number of images per block, per uid", "file": file}
        pbar.set_postfix(postfix)
        img_unique = df_img[~df_img['imageName_ImageInfo'].str.contains('fixation|grayCan|Message')]
        total_images = img_unique.groupby('blockNumber_ImageInfo')['imageName_ImageInfo'].nunique().reset_index().rename(columns={"blockNumber_ImageInfo": "block_number", "imageName_ImageInfo": "total_img"})

        # save total images for all participants
        if not os.path.isdir(t_path):
            os.mkdir(t_path)
        if not os.path.isdir(e_path):
            os.mkdir(e_path)

        total_images['uid'] = uid # pd.DataFrame(data={'uid':[uid], 'total_unique_img':[total_images]})
        if os.path.exists(total_img_file):
            df_total_unique_img = pd.read_csv(total_img_file)
            df_total_unique_img = pd.concat([df_total_unique_img,total_images], ignore_index=True)
            df_total_unique_img.reset_index(drop=True, inplace=True)
            df_total_unique_img.drop_duplicates(inplace=True)
        else:
            df_total_unique_img = total_images
        df_total_unique_img.to_csv(total_img_file, index=False)
        pbar.update(1)
        #### 5. Saving triggers and et files
        postfix = {"step": "5. Saving triggers and et files", "file": file}
        pbar.set_postfix(postfix)
        df_triggers.to_csv(os.path.join(t_path, 'trigger_file_' + uid +'.csv'), index=False)
        df_eye.to_csv(os.path.join(e_path, 'et_' + uid + '.csv'), index=False)
        pbar.update(1)
        pbar.set_postfix(file=file)
        pbar.close()

Processed:            0% 0/5 in 0.03s

:            0% 0/5 in 0.02s

:            0% 0/5 in 0.01s